In [1]:
! pip install tflearn

In [2]:
! pip install tqdm

In [3]:
! pip install tensorflow

In [4]:
import cv2
import numpy as np
import os         
from random import shuffle 
from tqdm import tqdm      
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline
TRAIN_DIR = "C:/Users/julia/Pictures/train"
TEST_DIR = "C:/Users/julia/Pictures/test"
IMG_SIZE = 50
LR = 1e-3
MODEL_NAME = 'dogs-vs-cats-convnet'

In [5]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat':return[1,0]
    elif word_label == 'dog':return[0,1]

In [6]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label=label_img(img)
        path=os.path.join(TRAIN_DIR, img)
        img=cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data
def create_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img_data = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [7]:
test_data = create_test_data()

100%|█████████████████████████████████████████████████████████████████████████████| 6000/6000 [00:08<00:00, 676.55it/s]


In [8]:
import numpy as np
import cv2
import os
from tqdm import tqdm
from random import shuffle

TRAIN_DIR = "C:/Users/julia/Pictures/train"
IMG_SIZE = 50

def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return [1, 0]
    elif word_label == 'dog': return [0, 1]

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue  # 防止讀不到圖檔時出錯
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    
    shuffle(training_data)

    # 分開圖片和標籤
    X = np.array([i[0] for i in training_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    Y = np.array([i[1] for i in training_data])

    # 儲存
    np.save('X.npy', X)
    np.save('Y.npy', Y)

    return X, Y

In [9]:
train_x, train_y = create_train_data()

100%|███████████████████████████████████████████████████████████████████████████| 11941/11941 [00:17<00:00, 682.81it/s]


In [26]:
train_data = create_train_data()

100%|███████████████████████████████████████████████████████████████████████████| 11941/11941 [00:17<00:00, 669.53it/s]


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

IMG_SIZE = 50
LR = 1e-3

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

IMG_SIZE = 50
LR = 1e-3

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=LR),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [29]:
MODEL_NAME = 'my_model.h5'

if os.path.exists(MODEL_NAME):
    model.load_weights(MODEL_NAME)
    print('Model loaded!')

In [30]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(train_data, test_size=0.2, random_state=42)

In [31]:
train = train_data[:-8000]
test = train_data[-8000:]

In [39]:
# 修正 test 資料，只留下合法圖片
test_clean = [i for i in test if isinstance(i[0], np.ndarray) and i[0].shape == (IMG_SIZE, IMG_SIZE)]

# 正確轉換
test_x = np.array([i[0] for i in test_clean]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = np.array([i[1] for i in test_clean])

In [58]:
print(img_data.shape)  # 看看現在的形狀
img_data = np.array(img_data)  # 確保它是 NumPy 陣列
img_data = img_data.squeeze()  # 去掉不必要的維度
test_clean = [i for i in test_data if isinstance(i[0], np.ndarray) and i[0].shape == (IMG_SIZE, IMG_SIZE)]

d = test_data[0]
img_data, img_num = d

data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
prediction = model.predict([data])[0]

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.imshow(img_data, cmap="gray")
print('Cat Possibility: {:5.2f}% \nDog Possibility: {:5.2f}%'.format(prediction[0]*100, prediction[1]*100))


()


ValueError: cannot reshape array of size 1 into shape (50,50,1)

In [55]:
fig = plt.figure(figsize=(16, 12))

for num, data in enumerate(test_data[:16]):
    img_num = data[1]
    img_data = data[0]
    
    # 🔧 修正可能的巢狀 array 結構
    img_data = np.array(img_data)
    if img_data.shape == (1, IMG_SIZE, IMG_SIZE):
        img_data = img_data[0]

    if img_data.shape != (IMG_SIZE, IMG_SIZE):
        print(f"Skipping invalid shape: {img_data.shape}")
        continue
    
    y = fig.add_subplot(4, 4, num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    data = data / 255.0  # 正規化

    model_out = model.predict(np.array([data]))[0]

    if np.argmax(model_out) == 1: 
        str_label = 'Dog'
    else:
        str_label = 'Cat'
        
    y.imshow(orig, cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)

plt.show()


Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()
Skipping invalid shape: ()


<Figure size 1600x1200 with 0 Axes>